<a href="https://colab.research.google.com/github/jpiche0720/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

# ASSIGNMENT


In [0]:
import category_encoders as ce
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import matplotlib.pyplot as plt

In [0]:
data = train.copy()

In [0]:
def clean(X):
    X = X.copy()
    
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    cols_with_zeros = ['longitude','latitude','gps_height','population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
   
    # Time Elapsed since construction
    X['construction_year'] = X['construction_year'].replace(0,np.NaN)
    X['construction_year'] = X['construction_year'].replace(np.NaN,round(X['construction_year'].mean()))
    X['elapsed_time'] = (pd.to_datetime(X['date_recorded']).dt.year -
                         pd.to_datetime(X['construction_year'],format='%Y').dt.year)
    
    # Extract dates and remove original columns
    X['date_recorded']= pd.to_datetime(X['date_recorded'])
    X['construction_year'] = pd.to_datetime(X['construction_year'],format='%Y').dt.year
    X['month'] = X['date_recorded'].dt.month
    X['day'] = X['date_recorded'].dt.day
    X['year'] = X['date_recorded'].dt.year               
    
    X = X.drop(columns='date_recorded')  
    
    
    
    # Drop Similar columns and unuseable variance 
    X = X.drop(columns=['payment_type','num_private','extraction_type_group',
                        'extraction_type_class','quantity_group','source_type',
                        'waterpoint_type_group','recorded_by'])
    
        
    high_cardinality = []
    for col in X.columns:
        if X[col].dtype == 'object' and X[col].nunique()>50:
            high_cardinality.append(col)
    
    print('Highly Cardinal Features = ', high_cardinality)
    return X

In [0]:
clean_data = clean(data)

Highly Cardinal Features =  ['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward', 'scheme_name']


In [0]:
train, val = train_test_split(clean_data, test_size=0.20,stratify=data['status_group'])
train.shape, val.shape

((47520, 36), (11880, 36))

In [0]:
target = 'status_group'
features = train.columns.drop([target,'id'])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]


pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(missing_values=np.NaN),
    RandomForestClassifier(random_state=42,n_jobs=-1))


param_dist = {
    'simpleimputer__strategy':['mean','median','most_frequent'],
    'randomforestclassifier__n_estimators':[100,200,300],
    'randomforestclassifier__max_depth':[3,5,7],
    'randomforestclassifier__min_samples_leaf':[3,5,8],
    'randomforestclassifier__min_samples_split':[2,3,4],
    'randomforestclassifier__max_features':[10,25,36]
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_dist, 
    n_iter=20, 
    cv=5, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

print(search.best_params_)
print(search.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.3min finished


{'simpleimputer__strategy': 'mean', 'randomforestclassifier__n_estimators': 200, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__min_samples_leaf': 10, 'randomforestclassifier__max_features': 10, 'randomforestclassifier__max_depth': 7}
0.7337962962962963


In [0]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,11,4,14,9,16,5,8,3,7,17,1,10,13,15,0,18,6,2,12,19
mean_fit_time,16.0696,17.1338,18.3844,39.9654,37.6001,17.5251,32.3773,27.7351,46.1872,13.702,7.58061,13.052,28.3236,2.51013,1.10874,2.30638,2.35443,1.50147,2.00959,0.892247
std_fit_time,0.856826,0.374537,2.38348,0.852057,2.00664,0.829444,8.68654,1.09196,7.50932,1.18284,1.30181,0.796281,1.80429,0.088294,0.0681186,0.364906,0.518796,0.0840174,0.372649,0.199536
mean_score_time,0.243378,0.227796,0.21048,0.248752,0.214844,0.269238,0.285175,0.28089,0.23779,0.213052,0.258479,0.233322,0.269101,0,0,0,0,0,0,0
std_score_time,0.0398073,0.0395199,0.00996107,0.0564842,0.00900838,0.053644,0.0684318,0.049028,0.0389196,0.013255,0.0568822,0.0400282,0.0531517,0,0,0,0,0,0,0
param_simpleimputer__strategy,mean,most_frequent,median,mean,most_frequent,mean,median,most_frequent,mean,mean,most_frequent,mean,median,most_frequent,most_frequent,most_frequent,most_frequent,most_frequent,mean,mean
param_randomforestclassifier__n_estimators,200,200,100,200,200,300,300,200,300,100,200,300,300,300,200,100,100,100,100,100
param_randomforestclassifier__min_samples_split,2,4,8,8,8,2,2,4,8,8,4,8,2,4,4,2,4,4,4,2
param_randomforestclassifier__min_samples_leaf,10,10,5,15,10,10,5,10,15,10,15,5,10,5,15,10,10,10,10,10
param_randomforestclassifier__max_features,10,10,25,25,25,10,10,25,25,25,10,10,25,36,36,36,36,36,36,36
param_randomforestclassifier__max_depth,7,7,7,7,7,5,5,5,5,5,3,3,3,5,5,3,5,7,7,3


In [0]:
pipeline = search.best_estimator_


train_score = pipeline.score(X_train,y_train)
val_score = pipeline.score(X_val,y_val)


print(f'Training accuracy = {train_score}')
print(f'Validation accuracy = {val_score}')

Training accuracy = 0.7370791245791246
Validation accuracy = 0.7379629629629629


In [0]:
target = 'status_group'
features = train.columns.drop([target,'id'])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]


pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(missing_values=np.NaN),
    RandomForestClassifier(n_estimators=200,
                           min_samples_leaf=3,
                           max_depth=25,
                           max_features=30,
                           random_state=42,
                           n_jobs=-1))

pipeline.fit(X_train,y_train)
train_score = pipeline.score(X_train,y_train)
val_score = pipeline.score(X_val,y_val)


print(f'Training accuracy = {train_score}')
print(f'Validation accuracy = {val_score}')

Training accuracy = 0.9400462962962963
Validation accuracy = 0.8079124579124579


In [0]:
features = train.columns.drop([target,'id'])
test = clean(test)
predictions = pipeline.predict(test[features])
print('Prediction.shape',predictions.shape)

col= test['id']
submission = pd.DataFrame(predictions,index=col,columns=['status_group'])

submission = submission.reset_index()
print(submission.shape)
submission.head()

Highly Cardinal Features =  ['funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward', 'scheme_name']
Prediction.shape (14358,)
(14358, 2)


,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [0]:
def csv_download_link(df, csv_file_name):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    df.to_csv(csv_file_name, index=False)
    from IPython.display import FileLink, FileLinks
    display(FileLink(csv_file_name))
# Diplay download link:
csv_download_link(submission, 'Tanzania.csv')



/Users/jp/DS-Unit-2-Kaggle-Challenge/module3-cross-validation/Tanzania.csv